In [1]:
import random

import tensorflow as tf

In [2]:
N = 100000
N_CLASSES = 50

In [3]:
def input_values():
    multiple_values = [map(int, '{0:050b}'.format(i)) 
                       for i in range(2**20)]
    random.shuffle(multiple_values)
    final_values = []
    for value in multiple_values[:N]:
        temp = []
        for i in value:
            temp.append([i])
        final_values.append(temp)
    return final_values

In [4]:
def output_values(inputs):
    final_values = []
    for value in inputs:
        output_values = [0 for _ in range(N_CLASSES)]
        count = 0
        for i in value:
            count += i[0]
        if count < N_CLASSES:
            output_values[count] = 1
        final_values.append(output_values)
    return final_values

In [5]:
def generate_data():
    inputs = input_values()
    return inputs, output_values(inputs)

In [6]:
X = tf.placeholder(tf.float32, shape=[None, N_CLASSES, 1])
Y = tf.placeholder(tf.float32, shape=[None, N_CLASSES])
N_HIDDEN = 24

weights = tf.Variable(tf.truncated_normal([N_HIDDEN, N_CLASSES]))
biases = tf.Variable(tf.truncated_normal([N_CLASSES]))

In [7]:
rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units=N_HIDDEN, 
                                       activation=tf.nn.relu)
outputs1, state = tf.nn.dynamic_rnn(rnn_cell, inputs=X, dtype=tf.float32)
outputs = tf.transpose(outputs1, [1, 0, 2])

In [8]:
last_output = tf.gather(outputs, int(outputs.get_shape()[0]) - 1)
prediction = tf.matmul(last_output, weights) + biases

In [9]:
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, 
                                                  logits=prediction)
total_loss = tf.reduce_mean(loss)

In [10]:
ETA = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate=ETA)\
    .minimize(loss=total_loss)

/usr/local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [31]:
BATCH_SIZE = 1000
N_BATCHES = int(N / BATCH_SIZE)
EPOCHS = 50

test_example = [[
    [0], [1], [1], [0], [0], [0], [1], [0], [0], [1],
    [1], [1], [1], [0], [1], [0], [0], [0], [1], [1],
    [1], [0], [0], [0], [1], [0], [1], [0], [1], [0],
    [1], [1], [1], [0], [1], [0], [0], [0], [1], [1],
    [1], [1], [1], [0], [1], [0], [0], [0], [1], [1]]]

In [32]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    X_train, y_train = generate_data()
    for epoch in range(EPOCHS):
        print('\n' + '-' * 60)
        print('Epoch:', epoch)
        i = 0
        for _ in range(N_BATCHES):
            training_x = X_train[i:i + BATCH_SIZE]
            training_y = y_train[i:i + BATCH_SIZE]
            i += BATCH_SIZE
            _, current_total_loss = sess.run(
                [optimizer, total_loss], 
                feed_dict={X: training_x, Y: training_y})
            print('Iter:', i, 'Loss:', current_total_loss, end='\r')
            
    pred_result = sess.run(prediction, {X: test_example})
    largest_number_index = pred_result[0].argsort()[-1:][::-1]
    print('Predicted sum:', largest_number_index, 'Actual:', 26)
    print('The predicted sequence parity is ', largest_number_index % 2, 
          'and it should be:', 0)


------------------------------------------------------------
Epoch: 0
Iter: 100000 Loss: 3.535387
------------------------------------------------------------
Epoch: 1
Iter: 100000 Loss: 2.16721532.7354205
------------------------------------------------------------
Epoch: 2
Iter: 100000 Loss: 1.7057124
------------------------------------------------------------
Epoch: 3
Iter: 100000 Loss: 1.3609849
------------------------------------------------------------
Epoch: 4
Iter: 100000 Loss: 1.2009692
------------------------------------------------------------
Epoch: 5
Iter: 100000 Loss: 1.1164911
------------------------------------------------------------
Epoch: 6
Iter: 100000 Loss: 1.2702043
------------------------------------------------------------
Epoch: 7
Iter: 100000 Loss: 0.8865079
------------------------------------------------------------
Epoch: 8
Iter: 100000 Loss: 0.8152359
------------------------------------------------------------
Epoch: 9
Iter: 100000 Loss: 0.7631535
-